In [1]:
from pysec import xbrl

In [46]:
from xbrl import XBRLParser, GAAP, GAAPSerializer
xbrl_parser = XBRLParser()
xbrl = xbrl_parser.parse(file("/Users/User/Documents/apty-20180131.xml"))
gaap_obj = xbrl_parser.parseGAAP(xbrl, doc_date="20131228", context="current", ignore_errors=0)
serializer = GAAPSerializer()
result = serializer.dump(gaap_obj)
print(result.data)

{u'liabilities': 0.0, u'net_cash_flows_financing_continuing': 0.0, u'revenue': 0.0, u'income_tax_expense_benefit': 0.0, u'common_shares_authorized': 0.0, u'income_from_equity_investments': 0.0, u'preferred_stock_dividends': 0.0, u'redeemable_noncontrolling_interest': 0.0, u'extraordary_items_gain_loss': 0.0, u'temporary_equity': 0.0, u'costs_and_expenses': 0.0, u'non_current_assets': 0.0, u'net_cash_flows_discontinued': 0.0, u'net_cash_flows_investing_discontinued': 0.0, u'liabilities_and_equity': 0.0, u'other_operating_income': 0.0, u'operating_income_loss': 0.0, u'income_before_equity_investments': 0.0, u'net_income_parent': 0.0, u'equity': 0.0, u'income_loss': 0.0, u'cost_of_revenue': 0.0, u'operating_expenses': 0.0, u'noncurrent_liabilities': 0.0, u'current_liabilities': 0.0, u'net_cash_flows_investing': 0.0, u'stockholders_equity': 0.0, u'net_income_loss': 0.0, u'net_cash_flows_investing_continuing': 0.0, u'nonoperating_income_loss': 0.0, u'common_shares_outstanding': 0.0, u'net_c

In [12]:
# Import requests to retrive Web Urls example HTML. TXT 
import requests

# Import BeautifulSoup
from bs4 import BeautifulSoup

# import re module for REGEXes
import re

# import pandas
import pandas as pd

In [47]:
r = requests.get('https://www.sec.gov/Archives/edgar/data/789019/000156459018019062/msft-10k_20180630.htm')
# r = requests.get('https://www.sec.gov/Archives/edgar/data/320193/000032019318000145/0000320193-18-000145.txt')
raw_10k = r.text

In [48]:
print(raw_10k[0:1300])

<DOCUMENT>
<TYPE>10-K
<SEQUENCE>1
<FILENAME>msft-10k_20180630.htm
<DESCRIPTION>10-K
<TEXT>
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<title>
msft-10k_20180630.htm
</title>
</head>
<!-- NG Converter v5.0.18157.137 -->
<body>

<p style="margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-family:Times New Roman;font-size:11pt;">&nbsp;</p>
<p style="text-align:center;margin-top:0pt;border-top:Solid 1pt #000000;padding-top:0pt;line-height:1pt;margin-bottom:0pt;text-indent:0%;font-size:1pt;">&nbsp;</p>
<p style="text-align:center;margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-weight:bold;font-size:17pt;font-family:Arial;font-style:normal;text-transform:none;font-variant: normal;">UNITED STATES </p>
<p style="text-align:center;margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-weight:bold;font-size:17pt;font-family:Arial;font-style:normal;text-transform:none;font-variant: normal;">SECURITIES AND EXCHANGE COMMI

In [49]:
# Regex to find <DOCUMENT> tags
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')

In [50]:
# Create 3 lists with the span idices for each regex

### There are many <Document> Tags in this text file, each as specific exhibit like 10-K, EX-10.17 etc
### First filter will give us document tag start <end> and document tag end's <start> 
### We will use this to later grab content in between these tags
doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]

### Type filter is interesting, it looks for <TYPE> with Not flag as new line, ie terminare there, with + sign
### to look for any char afterwards until new line \n. This will give us <TYPE> followed Section Name like '10-K'
### Once we have have this, it returns String Array, below line will with find content after <TYPE> ie, '10-K' 
### as section names
doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]

In [51]:
document = {}

# Create a loop to go through each section type and save only the 10-K section in the dictionary
for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    if doc_type == '10-K':
        document[doc_type] = raw_10k[doc_start:doc_end]

In [52]:
document['10-K'][0:500]

u'\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>msft-10k_20180630.htm\n<DESCRIPTION>10-K\n<TEXT>\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n<head>\n<title>\nmsft-10k_20180630.htm\n</title>\n</head>\n<!-- NG Converter v5.0.18157.137 -->\n<body>\n\n<p style="margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-family:Times New Roman;font-size:11pt;">&nbsp;</p>\n<p style="text-align:center;margin-top:0pt;border-top:Solid 1pt #000000;padding-top:0pt;line-hei'

In [53]:

# Write the regex
regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(1A|6|1B|7A|7|8)\.{0,1})|(ITEM\s(1A|6|1B|7A|7|8))')

# Use finditer to math the regex
matches = regex.finditer(document['10-K'])

# Write a for loop to print the matches
for match in matches:
    print(match)

In [54]:
# Matches
matches = regex.finditer(document['10-K'])

# Create the dataframe
test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

test_df.columns = ['item', 'start', 'end']
test_df['item'] = test_df.item.str.lower()

# Display the dataframe
test_df.head()

,item,start,end
0,>item&#160;1a.,31146,31160
1,>item&#160;1b.,34020,34034
2,>item&#160;6.,50774,50787
3,>item&#160;7.,53668,53681
4,>item&#160;7a.,56640,56654


In [55]:
# Get rid of unnesesary charcters from the dataframe
test_df.replace('&#160;',' ',regex=True,inplace=True)
test_df.replace('&nbsp;',' ',regex=True,inplace=True)
test_df.replace(' ','',regex=True,inplace=True)
test_df.replace('\.','',regex=True,inplace=True)
test_df.replace('>','',regex=True,inplace=True)

# display the dataframe
test_df.head()

,item,start,end
0,item1a,31146,31160
1,item1b,34020,34034
2,item6,50774,50787
3,item7,53668,53681
4,item7a,56640,56654


In [56]:
# Drop duplicates
pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')

# Display the dataframe
pos_dat

,item,start,end
17,item1a,357212,357220
18,item1b,364949,364957
19,item6,539344,539351
37,item7,1394815,1394822
38,item7a,1401202,1401210
85,item8,5883010,5883017


In [57]:
# Set item as the dataframe index
pos_dat.set_index('item', inplace=True)

# display the dataframe
pos_dat

,start,end
item,,
item1a,357212,357220
item1b,364949,364957
item6,539344,539351
item7,1394815,1394822
item7a,1401202,1401210
item8,5883010,5883017


In [63]:
# Get Item 1a
item_1a_raw = document['10-K'][pos_dat['start'].loc['item1a']:pos_dat['start'].loc['item1b']]

# Get Item 7
item_7_raw = document['10-K'][pos_dat['start'].loc['item7']:pos_dat['start'].loc['item7a']]

item_6_raw = document['10-K'][pos_dat['start'].loc['item6']:pos_dat['start'].loc['item7']]

# Get Item 7a
item_7a_raw = document['10-K'][pos_dat['start'].loc['item7a']:pos_dat['start'].loc['item8']]

In [64]:
item_6_raw[0:1000]

u'>Item 6</p>\n<p style="text-align:center;margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-size:9pt;">&nbsp;</p>\n<p style="text-align:center;margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-weight:bold;font-size:12pt;font-family:Arial;font-style:normal;text-transform:none;font-variant: normal;"><a name="ITEM_6_SELECTED_FINANCIAL_DATA"></a><font style="font-weight:bold;font-size:12pt;font-family:Arial;font-style:normal;text-transform:none;font-variant: normal;"><a name="ITEM_6_SELECTED_FINANCIAL_DATA"></a>ITEM&#160;6.&#160;SELECTED</font><font style="font-weight:bold;font-size:12pt;font-family:Arial;font-style:normal;text-transform:none;font-variant: normal;"> FINANCIAL DATA </font></p>\n<p style="text-align:center;margin-bottom:0pt;margin-top:9pt;text-indent:0%;font-size:10pt;font-family:Arial;font-weight:normal;font-style:normal;text-transform:none;font-variant: normal;"><font style="text-decoration:underline;">FINANCIAL HIGHLIGHTS </font></p>\n<p style="margin-bottom:0pt;ma

In [65]:
### First convert the raw text we have to exrtacted to BeautifulSoup object 
item_1a_content = BeautifulSoup(item_1a_raw, 'lxml')
item_6_content = BeautifulSoup(item_6_raw, 'lxml')

In [70]:
### By just applying .pretiffy() we see that raw text start to look oragnized, as BeautifulSoup
### apply indentation according to the HTML Tag tree structure
print(item_7_content.prettify()[0:1000])

<html>
 <body>
  <p>
   &gt;Item 7
  </p>
  <p style="text-align:center;margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-size:9pt;">
  </p>
  <p style="margin-bottom:0pt;margin-top:0pt;text-align:justify;text-indent:0%;font-weight:bold;font-size:10pt;font-family:Arial;font-style:normal;text-transform:none;font-variant: normal;">
   <font style="font-weight:bold;font-size:10pt;font-family:Arial;font-style:normal;text-transform:none;font-variant: normal;">
    STATEMENT OF MANAGEMENT’S RESPONSIBILITY FOR
   </font>
   <font style="font-weight:bold;font-size:10pt;font-family:Arial;font-style:normal;text-transform:none;font-variant: normal;">
    FINANCIAL STATEMENTS
   </font>
  </p>
  <p style="margin-bottom:0pt;margin-top:9pt;text-align:justify;text-indent:0%;font-size:10pt;font-family:Arial;font-weight:normal;font-style:normal;text-transform:none;font-variant: normal;">
   Management is responsible for the preparation of the consolidated financial statements and related information

In [71]:
### Our goal is though to remove html tags and see the content
### Method get_text() is what we need, \n\n is optional, I just added this to read text 
### more cleanly, it's basically new line character between sections. 
print(item_7_content.get_text("\n\n")[0:1500])

>Item 7




 




STATEMENT OF MANAGEMENT’S RESPONSIBILITY FOR

 FINANCIAL STATEMENTS 




Management is responsible for the preparation of the consolidated financial statements and related information that are presented in this report. The consolidated financial statements, which include amounts based on management’s estimates and judgments, have been prepared in conformity with accounting principles generally accepted in the United States of America. 




The Company designs and maintains accounting and internal control systems to provide reasonable assurance at reasonable cost that assets are safeguarded against loss from unauthorized use or disposition, and that the financial records are reliable for preparing consolidated financial statements and maintaining accountability for assets. These systems are augmented by written policies, an organizational structure providing division of responsibilities, careful selection and training of qualified personnel, and a program of internal a

**Section 2: Numerical Data**

ImportError: No module named edgar.stock